In [203]:
import pandas as pd
import numpy as np
import tensorflow as tf

np.set_printoptions(threshold=np.inf)

train_features = pd.read_csv(open("train_features.tsv"),sep='\t')
train_labels = pd.read_csv(open("train_labels.tsv"),sep='\t')


tags = np.array(train_features['tag'],dtype = np.str)
list1 = []
for i in range(tags.size):
    list1.extend(tags[i].split(','))
list2 = list(set(list1))
list2.sort()
tags1 = np.zeros((5240,200))
for i in range(5240):
    for j in range(200):
        tempList = tags[i].split(',')
        if list2[j] in tempList:
            tags1[i,j]=1
            

            
from sklearn.preprocessing import LabelEncoder
columnsToEncode = list(train_features.select_dtypes(include=['category','object']))
le = LabelEncoder()
for feature in columnsToEncode:
    try:
        train_features[feature] = le.fit_transform(train_features[feature].astype(str))
    except:
        print('Error encoding ' + feature)
        
columnsToEncode = list(train_labels.select_dtypes(include=['category','object']))
le = LabelEncoder()
for feature in columnsToEncode:
    try:
        train_labels[feature] = le.fit_transform(train_labels[feature].astype(str))
    except:
        print('Error encoding ' + feature)
        
temp_features = np.array(train_features)
temp_features = np.delete(temp_features,4,axis = 1)
for i in range(5240):
    temp_features[i,3] = temp_features[i,3]+1915
#print(temp_features[:,3])
new_features = np.append(tags1,temp_features[:,4:],axis = 1)

train_labels1 = np.array(train_labels)

In [204]:
valid_features = pd.read_csv(open("valid_features.tsv"),sep='\t')
valid_labels = pd.read_csv(open("valid_labels.tsv"),sep='\t')

valid_tags = np.array(valid_features['tag'],dtype = np.str)

valid_tags1 = np.zeros((299,200))
for i in range(299):
    for j in range(200):
        tempList = valid_tags[i].split(',')
        if list2[j] in tempList:
            valid_tags1[i,j]=1
            


from sklearn.preprocessing import LabelEncoder
columnsToEncode = list(valid_features.select_dtypes(include=['category','object']))
le = LabelEncoder()
for feature in columnsToEncode:
    try:
        valid_features[feature] = le.fit_transform(valid_features[feature].astype(str))
    except:
        print('Error encoding ' + feature)
        


columnsToEncode = list(valid_labels.select_dtypes(include=['category','object']))
le = LabelEncoder()
for feature in columnsToEncode:
    try:
        valid_labels[feature] = le.fit_transform(valid_labels[feature].astype(str))
    except:
        print('Error encoding ' + feature)
        
temp_valid_features = np.array(valid_features)
temp_valid_features = np.delete(temp_valid_features,4,axis = 1)
for i in range(299):
    temp_valid_features[i,3] = temp_valid_features[i,3]+1915

new_valid_features = np.append(valid_tags1,temp_valid_features[:,4:],axis = 1)

valid_labels1 = np.array(valid_labels)

In [205]:
test_features = pd.read_csv(open("test_features.tsv"),sep='\t')

test_tags = np.array(test_features['tag'],dtype = np.str)

test_tags1 = np.zeros((235,200))
for i in range(235):
    for j in range(200):
        tempList = test_tags[i].split(',')
        if list2[j] in tempList:
            test_tags1[i,j]=1

from sklearn.preprocessing import LabelEncoder
columnsToEncode = list(test_features.select_dtypes(include=['category','object']))
le = LabelEncoder()
for feature in columnsToEncode:
    try:
        test_features[feature] = le.fit_transform(test_features[feature].astype(str))
    except:
        print('Error encoding ' + feature)
        
        
temp_test_features = np.array(test_features)
temp_test_features = np.delete(temp_test_features,4,axis = 1)
for i in range(235):
    temp_test_features[i,3] = temp_test_features[i,3]+1915

new_test_features = np.append(test_tags1,temp_test_features[:,4:],axis = 1)

In [206]:
new_labels = np.zeros((5240,18))
for i in range(5240):
    for j in range(18):
        if train_labels1[i,1]==j:
            new_labels[i,j]=1

In [207]:
valid_labels111 = np.zeros((299,18))
for i in range(299):
    for j in range(18):
        if valid_labels1[i,1]==j:
            valid_labels111[i,j]=1

In [208]:
import math

N = 100 
D = 327 
num_label = 18 
num_data = 5240
hidden_size_1 = 40
hidden_size_2 = 40
hidden_size_3 = 80

beta = 0.001 
learning_rate = 0.001 

#labels = (np.arange(num_label) == y[:,None]).astype(np.float32)

graph = tf.Graph()
with graph.as_default():
    x = tf.constant(new_features.astype(np.float32))
    tf_labels = tf.constant(new_labels)
    
    # hidden layer 1
    hidden_layer_weights_1 = tf.Variable(
    tf.truncated_normal([D, hidden_size_1], stddev=math.sqrt(2.0/num_data)))
    hidden_layer_bias_1 = tf.Variable(tf.zeros([hidden_size_1]))
    
    # hidden layer 2
    hidden_layer_weights_2 = tf.Variable(
    tf.truncated_normal([hidden_size_1, hidden_size_2], stddev=math.sqrt(2.0/hidden_size_1)))
    hidden_layer_bias_2 = tf.Variable(tf.zeros([hidden_size_2]))
    '''    
    # hidden layer 2
    hidden_layer_weights_3 = tf.Variable(
    tf.truncated_normal([hidden_size_2, hidden_size_3], stddev=math.sqrt(2.0/hidden_size_2)))
    hidden_layer_bias_3 = tf.Variable(tf.zeros([hidden_size_3]))
   
    '''
    # output layer
    out_weights = tf.Variable(
    tf.truncated_normal([hidden_size_2, num_label], stddev=math.sqrt(2.0/hidden_size_2)))
    out_bias = tf.Variable(tf.zeros([num_label]))
    

    z1 = tf.matmul(x, hidden_layer_weights_1) + hidden_layer_bias_1
    h1 = tf.nn.relu(z1)
    
    z2 = tf.matmul(h1, hidden_layer_weights_2) + hidden_layer_bias_2
    h2 = tf.nn.relu(z2)
    """
    z3 = tf.matmul(h2, hidden_layer_weights_3) + hidden_layer_bias_3
    h3 = tf.nn.relu(z3)
    """
    logits = tf.matmul(h2, out_weights) + out_bias
    
    # L2 regularization
    regularization = tf.nn.l2_loss(hidden_layer_weights_1) + tf.nn.l2_loss(hidden_layer_weights_2)  + tf.nn.l2_loss(out_weights)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_labels, logits=logits) + beta * regularization) 
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_labels, logits=logits))
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    train_prediction = tf.nn.softmax(logits)

    weights = [hidden_layer_weights_1, hidden_layer_bias_1, hidden_layer_weights_2, hidden_layer_bias_2, out_weights, out_bias]
        
 

In [217]:
num_steps = 3000

def accuracy(predictions, labels):
    #print(predictions.shape[0])
    #print(np.argmax(labels, axis=1))
    return (100.0 * np.sum(np.argmax(predictions, axis=1) == np.argmax(labels, axis=1))
          / predictions.shape[0])

def relu(x):
    return np.maximum(0,x)
          

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        _, l, predictions = session.run([train_op, loss, train_prediction])
    
        if (step % 200 == 0):
            #print(predictions)
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, new_labels))
    w1, b1, w2, b2, w3, b3 = weights
    #print(w1)
    #print(w1.eval())
    
    #Z = np.dot(np.dot(np.dot(new_valid_features, w1.eval()) + b1.eval(), w2.eval()) + b2.eval(), w3.eval()) + b3.eval()
    Z = np.dot(relu(np.dot(relu(np.dot(new_valid_features, w1.eval()) + b1.eval()), w2.eval()) + b2.eval()), w3.eval()) + b3.eval()
    pred = np.dot(relu(np.dot(relu(np.dot(new_test_features, w1.eval()) + b1.eval()), w2.eval()) + b2.eval()), w3.eval()) + b3.eval()
    print('Validation accuracy: %.1f%%' % accuracy(Z, valid_labels111))
    

Initialized
Loss at step 0: 6.417058
Training accuracy: 4.7%
Loss at step 200: 2.153193
Training accuracy: 36.5%
Loss at step 400: 1.827898
Training accuracy: 45.4%
Loss at step 600: 1.716215
Training accuracy: 49.2%
Loss at step 800: 1.656546
Training accuracy: 51.4%
Loss at step 1000: 1.610652
Training accuracy: 52.9%
Loss at step 1200: 1.580900
Training accuracy: 54.0%
Loss at step 1400: 1.545078
Training accuracy: 55.3%
Loss at step 1600: 1.536815
Training accuracy: 55.6%
Loss at step 1800: 1.511133
Training accuracy: 56.9%
Loss at step 2000: 1.490168
Training accuracy: 57.0%
Loss at step 2200: 1.472742
Training accuracy: 58.0%
Loss at step 2400: 1.459695
Training accuracy: 59.0%
Loss at step 2600: 1.440844
Training accuracy: 59.8%
Loss at step 2800: 1.423917
Training accuracy: 61.2%
Validation accuracy: 39.1%


In [219]:
Z1=np.argmax(Z, axis=1)

P_R = np.zeros((18,4))
actual = valid_labels1[:,-1]
predicted = Z1.flatten()
num_of_each_grade = np.zeros((18,1)).flatten()
for i in range(18):
    sum1 = 0
    sum2 = 0
    sum3 = 0
    sum4 = 0
    sum_of_instance = 0
    for j in range(len(actual)):
        if actual[j]==predicted[j]==(i):
            sum1 = sum1+1
        if actual[j]==(i)!=predicted[j]:
            sum2 = sum2+1
        if predicted[j]==(i)!=actual[j]:
            sum3 = sum3+1
        if predicted[j]!=(i) and actual[j]!=(i):
            sum4 = sum4+1
        if actual[j]==(i):
            sum_of_instance = sum_of_instance+1
    num_of_each_grade[i] = sum_of_instance
    P_R[i,0] = sum1
    P_R[i,1] = sum2
    P_R[i,2] = sum3
    P_R[i,3] = sum4

closer_evaluation = np.zeros((18,3))
for i in range(18):
    if((P_R[i,0]+P_R[i,2])!=0):
        closer_evaluation[i,0]=P_R[i,0]/(P_R[i,0]+P_R[i,2])
    if((P_R[i,0]+P_R[i,1])!=0):
        closer_evaluation[i,1]=P_R[i,0]/(P_R[i,0]+P_R[i,1])
    P = closer_evaluation[i,0]
    R = closer_evaluation[i,1]
    if(P==0 and R==0):
        closer_evaluation[i,2]=0
    else:
        closer_evaluation[i,2]= 2*P*R/(P+R)
precision = 0
for i in range(18):
    precision = precision + closer_evaluation[i,0]
precision = precision/18

recall = 0
for i in range(18):
    recall = recall + closer_evaluation[i,1]
recall = recall/18

fscore = 2*precision*recall/(precision+recall)

print("Neural Network precision(macro):\n"+'%.2f%%'%(precision*100))
print("Neural Network recall(macro):\n"+'%.2f%%'%(recall*100))
print("Neural Network F-Score(macro):\n"+'%.2f%%'%(fscore*100))



Neural Network precision(macro):
37.27%
Neural Network recall(macro):
30.41%
Neural Network F-Score(macro):
33.49%


In [216]:
pred1=np.argmax(pred, axis=1)

#pred2 = np.ones((235,1), np.dtype = np.string)

a = np.ones((235,1)) 
b = a.astype(np.str)
for i in range(235):
    if pred1[i]==0:
        b[i,0]='Action'
    if pred1[i]==1:
        b[i,0]='Adventure'
    if pred1[i]==2:
        b[i,0]='Animation'
    if pred1[i]==3:
        b[i,0]='Children'
    if pred1[i]==4:
        b[i,0]='Comedy'
    if pred1[i]==5:
        b[i,0]='Crime'
    if pred1[i]==6:
        b[i,0]='Documentary'
    if pred1[i]==7:
        b[i,0]='Drama'
    if pred1[i]==8:
        b[i,0]='Fantasy'
    if pred1[i]==9:
        b[i,0]='Film_Noir'
    if pred1[i]==10:
        b[i,0]='Horror'
    if pred1[i]==11:
        b[i,0]='Musical'
    if pred1[i]==12:
        b[i,0]='Mystery'
    if pred1[i]==13:
        b[i,0]='Romance'
    if pred1[i]==14:
        b[i,0]='Sci_Fi'
    if pred1[i]==15:
        b[i,0]='Thriller'
    if pred1[i]==16:
        b[i,0]='War'
    if pred1[i]==17:
        b[i,0]='Western'
movieid = np.array(temp_test_features[:,:1],dtype=np.int)
no_head_final_result = np.append(movieid,b,axis = 1)
head = np.array([['movieId','genres']])
final_result=np.append(head,no_head_final_result,axis = 0)
final_result

array([['movieId', 'genres'],
       ['91511', 'Comedy'],
       ['91535', 'Fantasy'],
       ['91542', 'Crime'],
       ['91797', 'Drama'],
       ['91826', 'Drama'],
       ['91886', 'Drama'],
       ['91888', 'Animation'],
       ['91911', 'Drama'],
       ['91976', 'Romance'],
       ['92198', 'Thriller'],
       ['92206', 'Drama'],
       ['92210', 'Fantasy'],
       ['92224', 'Drama'],
       ['92234', 'War'],
       ['92243', 'War'],
       ['92352', 'Drama'],
       ['92391', 'Horror'],
       ['92475', 'War'],
       ['92507', 'Thriller'],
       ['92652', 'Documentary'],
       ['92768', 'Drama'],
       ['93132', 'Children'],
       ['93181', 'Documentary'],
       ['93183', 'Drama'],
       ['93196', 'Comedy'],
       ['93344', 'Sci_Fi'],
       ['93367', 'Mystery'],
       ['93475', 'Thriller'],
       ['93496', 'Romance'],
       ['93520', 'Romance'],
       ['93563', 'Sci_Fi'],
       ['93570', 'Drama'],
       ['93831', 'Romance'],
       ['93840', 'Horror'],
       ['9

In [215]:

np.savetxt("final_result.csv", final_result, fmt='%s',delimiter=',')